# Pinecone_Setup & Sample Search Pipeline

## Step 1: Set Up Pinecone API

In [ ]:
# Step 2: Import libraries
from pinecone import Pinecone, ServerlessSpec
from sentence_transformers import SentenceTransformer
import os

# Step 3: Initialize Pinecone
api_key = "your_api_pinecone"  # 🔐 Paste your Pinecone API key
pc = Pinecone(api_key=api_key)

# Step 4: Create index (only do this once, skip if already created)
index_name = "nlp-demo"
dimension = 384  # because we're using all-MiniLM-L6-v2

if index_name not in pc.list_indexes().names():
    pc.create_index(
        name=index_name,
        dimension=dimension,
        metric="cosine",
        spec=ServerlessSpec(
            cloud="aws",        # or "gcp"
            region="us-east-1"  # match your Pinecone project region
        )
    )

In [2]:
# Step 5: Load your model and generate embeddings
model = SentenceTransformer("all-MiniLM-L6-v2")

sentences = [
    "Looking for a backend engineer with Python experience.",
    "Proficient in data analysis using SQL and Python.",
    "Seeking an expert in AWS cloud deployment.",
]

embeddings = model.encode(sentences)

# Step 6: Format and upsert into Pinecone
index = pc.Index(index_name)
to_upsert = [(f"id-{i}", embedding.tolist()) for i, embedding in enumerate(embeddings)]

index.upsert(vectors=to_upsert)

{'upserted_count': 3}

In [3]:
# Query sentence
query = "Cloud engineer with AWS knowledge"
query_vector = model.encode([query]).tolist()[0]

# Perform search
result = index.query(vector=query_vector, top_k=3)

# Sentence mapping (for display purposes)
id_to_sentence = {
    f"id-{i}": sentence for i, sentence in enumerate(sentences)
}

# Display results with full sentence
print(f"\n🔍 Query: {query}\n")
for match in result['matches']:
    matched_id = match['id']
    matched_score = match['score']
    matched_sentence = id_to_sentence.get(matched_id, "Sentence not found")

    print(f"✅ Match: {matched_sentence}")
    print(f"🆔 ID: {matched_id} | 🔢 Score: {matched_score:.4f}\n")


🔍 Query: Cloud engineer with AWS knowledge

✅ Match: Seeking an expert in AWS cloud deployment.
🆔 ID: id-2 | 🔢 Score: 0.7948

✅ Match: Looking for a backend engineer with Python experience.
🆔 ID: id-0 | 🔢 Score: 0.3860

✅ Match: Proficient in data analysis using SQL and Python.
🆔 ID: id-1 | 🔢 Score: 0.2042

